In [24]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
import json

load_dotenv()

True

In [25]:
#load in the generated story
with open("output/gpt-4o-mini/generated_story_level_2.txt", "r") as file:
    generated_story = file.read()

print(generated_story)

### Rapunzel

Once there was a man and a woman who wanted a baby. They wished for a long time. One day, the woman saw a garden from her window. It was full of beautiful flowers and herbs, but it had a high wall. The garden belonged to a powerful enchantress, and no one dared to go in.

The woman saw a plant called rampion in the garden. It looked so fresh and green that she really wanted to eat it. She felt so sick without it. Her husband noticed and asked, "What is wrong, dear wife?" 

"I want some rampion from the garden," she replied. "If I can't eat it, I will feel very sad!"

The husband cared for her and thought, "I will bring her some rampion." That night, he climbed over the wall and picked some. He gave it to his wife, and she made a salad. It tasted so good that she wanted more the next day.

The husband went back to the garden to get more rampion, but this time he saw the enchantress. She was angry. "How dare you steal my rampion?" she yelled. The husband was scared but expl

In [65]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
import json

load_dotenv()

class Clothing(BaseModel):
    clothing_type: str
    clothing_style: str
    clothing_colour: str
    clothing_pattern: str
    clothing_texture: str

class CharacterImagePrompt(BaseModel):
    name: str
    pose: str
    clothing: Clothing
    expression: str
    overall_description: str

class StoryImagePrompts(BaseModel):
    background_image_prompt: str
    character_image_prompts: list[CharacterImagePrompt]

class Scene(BaseModel):
    scene_no: int
    story_text: str
    story_image_prompts: StoryImagePrompts

class CharacterPermanentAttributes(BaseModel):
    name: str
    gender: str
    facial_features: str
    hair_colour: str
    hair_style: str
    body_type: str
    height: str
    weight: str
    species: str

class StoryScenes(BaseModel):
    illustration_style: str
    characters: list[CharacterPermanentAttributes]
    scenes: list[Scene]

def split_story_into_scenes(story_text: str) -> StoryScenes:
    # Define a prompt to instruct the model to split the story into scenes
    prompt = f"Split the following story into scenes. Separate each scene with a new line. This is the story: {story_text}"

    # Call the OpenAI API
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""You are an expert story illustrator creating a web app for children ages between 6-8 learning to read through stories. You are given a story text and before illustrating it, you need to 
                1. Read through the story text and chunk up the story into scenes. MAKE sure to have all the key scenes including the introduction, development, conflict, climax and resolution.
                2. Choose an illustration style for the story suitable for children ages between 6-8. Describe the style in detail so that can be used to generate consistent background images and characters. Describe the time period of the story, for example, for example, medieval, modern, futuristic, etc.
                3. Make a list of all the characters that appear in the story. If the same character is referred to with different ways, eg. man, husband, Mike, etc. you should refer to this character using his actual name if it is mentioned in the story, else refer to this character using the name that is most commonly used in the story and which is easily understood by someone reading your output. ALWAYS maintain consistency when referring to the same character throughout your output.
                3. For each scene, describe the background scene in detail, including the
                    - setting, 
                    - time of day
                    - weather
                    - the mood.
                    DO NOT INCLUDE any characters in the background description.
                4. For each scene, describe the characters in detail including their 
                    - clothing (include the type of clothes, composition, style, colour, texture, pattern, etc.)
                    - poses
                    - facial expressions 
                    so that it can be used to generate consistent images. The clothing description should be consistent for the same character unless it makes sense to change the character's clothing. Clothing description should include the type of clothes, composition, style, colour, texture, pattern, etc.              
                DO NOT alter the story text in any way.
                MAKE SURE TO use ALL the text provided in the story text."""
            },
            {
                "role": "user",
                "content": f"This is the story: {story_text}"
            }
        ],
        response_format=StoryScenes # Use "json" as the response format
    )

    # Parse the response into the StoryScenes model

    scenes = completion.choices[0].message.parsed
    return scenes
    
# Example usage
# scenes = split_story_into_scenes("Your story text here")
# print(scenes)

In [66]:
scenes = split_story_into_scenes(generated_story)
print(scenes)

illustration_style='The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright tones to capture the attention of children aged 6-8. The time period is medieval, with elements such as castles, towers, and enchanting forests that give the story a timeless fairy tale feel.' characters=[CharacterPermanentAttributes(name='The Woman', gender='Female', facial_features='Gentle face with soft features', hair_colour='Brown', hair_style='Long and wavy', body_type='Slim', height='Average', weight='Average', species='Human'), CharacterPermanentAttributes(name='The Husband', gender='Male', facial_features='Strong jawline with a warm s

In [67]:
scenes_json = scenes.model_dump(mode="json")
print(json.dumps(scenes_json, indent=4))

{
    "illustration_style": "The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright tones to capture the attention of children aged 6-8. The time period is medieval, with elements such as castles, towers, and enchanting forests that give the story a timeless fairy tale feel.",
    "characters": [
        {
            "name": "The Woman",
            "gender": "Female",
            "facial_features": "Gentle face with soft features",
            "hair_colour": "Brown",
            "hair_style": "Long and wavy",
            "body_type": "Slim",
            "height": "Average",
            "weight": "Average",
        

In [68]:
#write the output to file
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "w") as file:
    json.dump(scenes_json, file, indent=4)

In [69]:
#read the file storylabs/backend/output/gpt-4o-mini/generated_story_scenes_level_2.json
import json
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "r") as file:
    scenes_json = json.load(file)




## Flux

In [31]:
!poetry add replicate

The following packages are already present in the pyproject.toml and will be skipped:

  - replicate

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


In [32]:
import replicate
load_dotenv()
REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]

In [70]:
#load the image prompts from file
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "r") as file:
    story_image_prompts = json.load(file)

In [71]:
#load the list of characters
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "r") as file:
    story_image_prompts = json.load(file)
    characters = [story_image_prompts["characters"][i]["name"] for i in range(len(story_image_prompts["characters"]))]
    print(characters)

['The Woman', 'The Husband', 'The Enchantress', 'Rapunzel', 'The Prince']


In [109]:
character_name=characters[4]
print(character_name)

The Prince


In [110]:
#get all the clothing for character Woman
character_clothing = []
for scene in story_image_prompts["scenes"]:
    if scene['story_image_prompts'].get('character_image_prompts'):
        for character in scene['story_image_prompts']['character_image_prompts']:
            if character.get('name', '') == character_name:
                character_clothing.append(character['clothing'])

print(character_clothing)

[{'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}, {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}, {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}, {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}, {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}, {'clothing_type': '

In [111]:
character_permanent_attributes = next((character for character in story_image_prompts["characters"] if character["name"] == character_name), None)
print(character_permanent_attributes)


{'name': 'The Prince', 'gender': 'Male', 'facial_features': 'Chiseled features with a charming smile', 'hair_colour': 'Blonde', 'hair_style': 'Medium length and neat', 'body_type': 'Fit', 'height': 'Tall', 'weight': 'Medium', 'species': 'Human'}


In [112]:
#generate a trigger word for this character, random 5 letters
import random
import string
trigger_word = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
#concatenate with gender
print(trigger_word)

wL5W7


In [113]:
#1 image
image_prompt_single_image = f"""
Character permanent attributes: {character_permanent_attributes} 
Clothing: {character_clothing[0]}
Illustration style: {story_image_prompts['illustration_style']}
"""

#remove name from the image prompt text




print(image_prompt_single_image)




Character permanent attributes: {'name': 'The Prince', 'gender': 'Male', 'facial_features': 'Chiseled features with a charming smile', 'hair_colour': 'Blonde', 'hair_style': 'Medium length and neat', 'body_type': 'Fit', 'height': 'Tall', 'weight': 'Medium', 'species': 'Human'} 
Clothing: {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}
Illustration style: The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright tones to capture the attention of children aged 6-8. The time pe

In [132]:

#use openai to convert this prompt into prose for flux input prompt
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""write an image prompt for the character in the image prompt below. {image_prompt_single_image} DO NOT include the words Image Prompt"""
        }
    ]
)   


In [133]:
flux_prompt = completion.choices[0].message.content
print(flux_prompt)


Create an illustration of **The Prince**, a tall and fit young man with chiseled features and a charming smile. His medium-length blonde hair is neat and well-styled. Dress him in exquisite royal attire that embodies medieval elegance, featuring a navy blue and gold color scheme with subtle floral patterns and rich velvet texture. The setting should be whimsical, filled with lush greenery, enchanting gardens, and magical towers that evoke a fairy tale atmosphere. The illustration should capture the cheerful, inviting style, with vibrant colors and soft lines, ensuring to emphasize The Prince's exaggerated expressions that convey warmth and charm, appealing to a young audience of ages 6-8.


In [114]:
#character sheet
image_prompt_character_sheet = f"""
Generate a character sheet of the character below shown 5 different angles:

Permanent attributes: {character_permanent_attributes} 
Clothing: {character_clothing[0]}
Illustration style: {story_image_prompts['illustration_style']}
"""

#remove name from the image prompt text



image_prompt_text = image_prompt_character_sheet.replace(character_name, trigger_word)
print(image_prompt_text)


Generate a character sheet of the character below shown 5 different angles:

Permanent attributes: {'name': 'wL5W7', 'gender': 'Male', 'facial_features': 'Chiseled features with a charming smile', 'hair_colour': 'Blonde', 'hair_style': 'Medium length and neat', 'body_type': 'Fit', 'height': 'Tall', 'weight': 'Medium', 'species': 'Human'} 
Clothing: {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}
Illustration style: The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright to

In [ ]:
#write a character_name_trigger_word.txt with the
# Permanent attributes: {character_permanent_attributes} 
#Clothing: {character_clothing[0]} 



In [134]:
input = {
    "prompt": flux_prompt,
    "prompt_upsampling": False, 
    "aspect_ratio": "16:9", 
    "output_format": "png", 
    "seed": 123457
}

output = replicate.run(
    "black-forest-labs/flux-1.1-pro",
    input=input
)


In [135]:
#make output directory if it doesn't exist
os.makedirs("output/images", exist_ok=True)
#increment instead of keep appending _1
filename = f"output/images/{character_name}_{trigger_word}.png"
counter = 1
while os.path.exists(filename):
    filename = f"output/images/{character_name}_{trigger_word}_{counter}.png"
    counter += 1
with open(filename, "wb") as file:
    file.write(output.read())


# Generate more images of the same character given 1 starting image

In [98]:
import replicate
load_dotenv()
REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]

In [120]:
image_prompt_single_image

"\nCharacter permanent attributes: {'name': 'The Prince', 'gender': 'Male', 'facial_features': 'Chiseled features with a charming smile', 'hair_colour': 'Blonde', 'hair_style': 'Medium length and neat', 'body_type': 'Fit', 'height': 'Tall', 'weight': 'Medium', 'species': 'Human'} \nClothing: {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}\nIllustration style: The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright tones to capture the attention of children aged 6-8. The tim

In [136]:
image = open("output/images/ThePrince_wL5W7.png", "rb")
print(image)

<_io.BufferedReader name='output/images/ThePrince_wL5W7.png'>


In [137]:
print(flux_prompt)

Create an illustration of **The Prince**, a tall and fit young man with chiseled features and a charming smile. His medium-length blonde hair is neat and well-styled. Dress him in exquisite royal attire that embodies medieval elegance, featuring a navy blue and gold color scheme with subtle floral patterns and rich velvet texture. The setting should be whimsical, filled with lush greenery, enchanting gardens, and magical towers that evoke a fairy tale atmosphere. The illustration should capture the cheerful, inviting style, with vibrant colors and soft lines, ensuring to emphasize The Prince's exaggerated expressions that convey warmth and charm, appealing to a young audience of ages 6-8.


In [121]:
print(image_prompt_single_image)


Character permanent attributes: {'name': 'The Prince', 'gender': 'Male', 'facial_features': 'Chiseled features with a charming smile', 'hair_colour': 'Blonde', 'hair_style': 'Medium length and neat', 'body_type': 'Fit', 'height': 'Tall', 'weight': 'Medium', 'species': 'Human'} 
Clothing: {'clothing_type': 'Royal attire', 'clothing_style': 'Medieval with elegant cuts', 'clothing_colour': 'Navy blue and gold', 'clothing_pattern': 'Subtle floral', 'clothing_texture': 'Rich velvet'}
Illustration style: The illustration style for this story features vibrant colors and whimsical shapes reminiscent of classic fairy tales. Characters will be depicted in a charming, cartoonish manner with exaggerated expressions to convey emotions effectively. The backgrounds will incorporate elements of fantasy, with lush greenery, magical towers, and enchanting gardens. The overall aesthetic is cheerful and inviting, using soft lines and bright tones to capture the attention of children aged 6-8. The time pe

In [149]:
import requests

def download_image(url, save_path):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open a local file in binary write mode
        with open(save_path, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"Image successfully downloaded: {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")



In [144]:

output = replicate.run(
    "fofr/consistent-character:9c77a3c2f884193fcee4d89645f02a0b9def9434f9e03cb98460456b831c8772",
    input={
        "seed": 123456,
        "prompt": flux_prompt,
        "subject": image,
        "output_format": "png",
        "output_quality": 80,
        "negative_prompt": "",
        "randomise_poses": True,
        "number_of_outputs": 5,
        "number_of_images_per_pose": 1
    }
)

# The fofr/consistent-character model can stream output as it's running.
# The predict method returns an iterator, and you can iterate over that output.


In [145]:
fofr_images = []

for item in output:
    print(item)
    fofr_images.append(item)


https://replicate.delivery/xezq/I0gdtvUxpvbqCpScofidiuxEaoF7cdU3JQ35BD8eBvihRq5TA/ComfyUI_00001_.png
https://replicate.delivery/xezq/0UNZGeB5DMzvTSuHKvD6udpooqUdQYvUld3YMguIkSR1I18JA/ComfyUI_00002_.png
https://replicate.delivery/xezq/lG2fP3Co81WaVykpTnepStpWkGL4YtMHp6O1Rb4cWxc0Rq5TA/ComfyUI_00003_.png
https://replicate.delivery/xezq/8f52w1ShmdXFdaatU8w8jAgAtQy4vR44jToQESopGu3AJ18JA/ComfyUI_00004_.png
https://replicate.delivery/xezq/efVdbEL3e3IlSIC1XmTReOdwzedvWRLLkMFMDvJTyLkoRSNfE/ComfyUI_00005_.png


In [157]:
import requests

def download_image(url, save_path):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open a local file in binary write mode
        with open(save_path, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"Image successfully downloaded: {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

# URL of the image
image_url = "https://replicate.delivery/xezq/efVdbEL3e3IlSIC1XmTReOdwzedvWRLLkMFMDvJTyLkoRSNfE/ComfyUI_00005_.png"

# Local path where the image will be saved
save_path = "ComfyUI_00005_.png"

# Download the image
download_image(image_url, save_path)

Image successfully downloaded: ComfyUI_00005_.png


In [151]:
#makedir output/images/fofr/
os.makedirs("output/images/fofr/", exist_ok=True)
counter=0
for image in fofr_images:
    image_url = image
    save_path = "output/images/fofr/" + character_name + "_" + trigger_word + "_" + str(counter) + ".png"
    counter += 1
# Download the image
download_image(image_url, save_path)
#https://replicate.delivery/xezq/I0gdtvUxpvbqCpScofidiuxEaoF7cdU3JQ35BD8eBvihRq5TA/ComfyUI_00001_.png


NameError: name 'fofr_images' is not defined

In [147]:
training = replicate.trainings.create(
  destination="chengyongyeo/storylabs-flux-rapunzel",
  version="ostris/flux-dev-lora-trainer:e440909d3512c31646ee2e0c7d6f6f4923224863a6a10c494606e79fb5844497",
  input={
    "steps": 1000,
    "lora_rank": 16,
    "optimizer": "adamw8bit",
    "batch_size": 1,
    "resolution": "512,768,1024",
    "autocaption": True,
    "input_images": "output/images/TheWoman_3AGnA_2.png.zip",
    "trigger_word": trigger_word,
    "learning_rate": 0.0004,
    "wandb_project": "flux_train_replicate",
    "wandb_save_interval": 100,
    "caption_dropout_rate": 0.05,
    "cache_latents_to_disk": False,
    "wandb_sample_interval": 100
  }
)

ReplicateError: ReplicateError Details:
status: 403
detail: You do not have access to the specified training destination

In [102]:
!poetry add opencv-python

Using version ^4.10.0.84 for opencv-python

Updating dependencies
Resolving dependencies... (1.6s)

Package operations: 2 installs, 0 updates, 0 removals

  - Installing numpy (2.2.0): Pending...
  - Installing numpy (2.2.0): Downloading... 0%
  - Installing numpy (2.2.0): Downloading... 10%
  - Installing numpy (2.2.0): Downloading... 20%
  - Installing numpy (2.2.0): Downloading... 30%
  - Installing numpy (2.2.0): Downloading... 40%
  - Installing numpy (2.2.0): Downloading... 50%
  - Installing numpy (2.2.0): Downloading... 60%
  - Installing numpy (2.2.0): Downloading... 70%
  - Installing numpy (2.2.0): Downloading... 80%
  - Installing numpy (2.2.0): Downloading... 90%
  - Installing numpy (2.2.0): Downloading... 100%
  - Installing numpy (2.2.0): Installing...
  - Installing numpy (2.2.0)
  - Installing opencv-python (4.10.0.84): Pending...
  - Installing opencv-python (4.10.0.84): Downloading... 0%
  - Installing opencv-python (4.10.0.84): Downloading... 5%
  - Installing open

In [103]:
import cv2
import numpy as np
import os

def load_yolo_model():
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

def detect_objects(image, net, output_layers):
    height, width, channels = image.shape
    # Create a blob and perform a forward pass
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # Initialize lists for detected bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    class_ids = []

    # Iterate over each detection
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression to remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return [(boxes[i], confidences[i], class_ids[i]) for i in indices.flatten()]

def save_cropped_objects(image, detections, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, (box, confidence, class_id) in enumerate(detections):
        x, y, w, h = box
        crop_img = image[y:y+h, x:x+w]
        cv2.imwrite(f"{output_folder}/object_{i}.png", crop_img)
        print(f"Saved {output_folder}/object_{i}.png")

# Example usage
image_path = "path/to/character_sheet.png"
output_folder = "output/objects"
image = cv2.imread(image_path)

net, output_layers = load_yolo_model()
detections = detect_objects(image, net, output_layers)
save_cropped_objects(image, detections, output_folder)

[ WARN:0@0.027] global loadsave.cpp:241 findDecoder imread_('path/to/character_sheet.png'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'
